In [601]:
from tmrl import get_environment
from time import sleep
from math import floor, sqrt
#from tmrl.custom.custom_models import conv2d_out_dims, num_flat_features, mlp
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tmrl.config.config_constants as cfg
from collections import deque, namedtuple

import random


Hyperparameters and Device

In [602]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the network
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


CNN Model for Training

In [603]:
# Model paramters
image_height = 64  #height of images
image_width = 64   #width of images
num_images = 4  #number of images (tmrl returns 4 to us)
keep_prob = 0.5     

# Critic (takes state and actions)

In [604]:
class CNN_Trackmania_Critic(nn.Module):
    def __init__(self, seed):
        super(CNN_Trackmania_Critic, self).__init__()

        self.IMAGE_HEIGHT = image_height
        self.IMAGE_WIDTH = image_width
        self.keep_prob = keep_prob
        self.seed = torch.manual_seed(seed)

        # convolutional layers
        self.conv1 = nn.Conv2d(num_images, 24, kernel_size=8, stride=2)
        self.conv2 = nn.Conv2d(24, 36, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(36, 48, kernel_size=4, stride=2)
        self.conv4 = nn.Conv2d(48, 64, kernel_size=4, stride=2)

        # dropout layer
        self.dropout = nn.Dropout(keep_prob)

        # fully-connected layers, add 3 more for action
        self.fc1 = nn.Linear(64 * 1 * 1 + 12, 256)
        self.fc2 = nn.Linear(256, 100)
        self.fc3 = nn.Linear(100, 10)
        self.fc4 = nn.Linear(10, 3)

    def forward(self, x, action):
        speed, gear, rpm, images, act1, act2 = x
        images = torch.from_numpy(images).float().to(device)
        speed = torch.from_numpy(speed).to(device)
        gear = torch.from_numpy(gear).to(device)
        rpm = torch.from_numpy(rpm).to(device)
        act1 = torch.from_numpy(act1).to(device)
        act2 = torch.from_numpy(act2).to(device)
        #print(action)
        action = torch.from_numpy(action).float().to(device)

        x = F.elu(self.conv1(images))
        x = F.elu(self.conv2(x))
        x = F.elu(self.conv3(x))
        x = F.elu(self.conv4(x))

        #x = self.dropout(x)

        x = x.view(-1)
        act1 = act1.view(-1)
        act2 = act2.view(-1)
        action = action.view(-1)
        #print(x.size())
        x = torch.cat((speed, gear, rpm, x, act1, act2, action), -1)
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = self.fc4(x)

        return x

# Actor (takes only state)

In [605]:
class CNN_Trackmania_Actor(nn.Module):
    def __init__(self, seed):
        super(CNN_Trackmania_Actor, self).__init__()

        self.IMAGE_HEIGHT = image_height
        self.IMAGE_WIDTH = image_width
        self.keep_prob = keep_prob
        self.seed = torch.manual_seed(seed)

        # convolutional layers
        self.conv1 = nn.Conv2d(num_images, 24, kernel_size=8, stride=2)
        self.conv2 = nn.Conv2d(24, 36, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(36, 48, kernel_size=4, stride=2)
        self.conv4 = nn.Conv2d(48, 64, kernel_size=4, stride=2)

        # dropout layer
        self.dropout = nn.Dropout(keep_prob)

        # fully-connected layers
        self.fc1 = nn.Linear(64 * 1 * 1 + 9, 256)
        self.fc2 = nn.Linear(256, 100)
        self.fc3 = nn.Linear(100, 10)
        self.fc4 = nn.Linear(10, 3)

    def forward(self, x):
        speed, gear, rpm, images, act1, act2 = x
        images = torch.from_numpy(images).float().to(device)
        speed = torch.from_numpy(speed).to(device)
        gear = torch.from_numpy(gear).to(device)
        rpm = torch.from_numpy(rpm).to(device)
        act1 = torch.from_numpy(act1).to(device)
        act2 = torch.from_numpy(act2).to(device)

        x = F.elu(self.conv1(images))
        x = F.elu(self.conv2(x))
        x = F.elu(self.conv3(x))
        x = F.elu(self.conv4(x))

        #x = self.dropout(x)

        x = x.view(-1)
        act1 = act1.view(-1)
        act2 = act2.view(-1)

        #print(x.size())
        x = torch.cat((speed, gear, rpm, x, act1, act2), -1)
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = self.fc4(x)

        return x

In [606]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = np.vstack([e.state for e in experiences if e is not None])
        actions = np.vstack([e.action for e in experiences if e is not None])
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = np.vstack([e.next_state for e in experiences if e is not None])
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

Agent - Used from HW7 for a DeepQNetwork

In [607]:
class CNNAgent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Actor and critic
        self.actor = CNN_Trackmania_Actor(seed).to(device)
        self.actor_target = CNN_Trackmania_Actor(seed).to(device)
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=LR)

        # critic
        self.critic = CNN_Trackmania_Critic(seed).to(device)
        self.critic_target = CNN_Trackmania_Critic(seed).to(device)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)
        
        '''stop coding here'''
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        #state = torch.from_numpy(state).unsqueeze(0).to(device)
        self.actor.eval()
        with torch.no_grad():
            action_values = self.actor.forward(state)
        self.actor.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return action_values.cpu().detach().numpy()
        else:
            return [random.uniform(-1,1),random.uniform(-1,1),random.uniform(-1,1)]

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        # Obtain random minibatch of tuples from D
        
        ## Compute and minimize the loss
        ### Extract next maximum estimated value from target network
        #print("learning")
        #print(experiences)
        #Experiences[0] = state experiences[1] = state
        q_targets_next = []
        q_currents = []
        q_critics = []
        states, actions, rewards, nexts, dones = experiences[0], experiences[1], experiences[2], experiences[3], experiences[4]
        for i in range(BATCH_SIZE): #compute Q loss for critic (state has actions already in it, actor ignores the actions)
            state_e, action_e, reward_e, next_e, done_e = states[i], actions[i], rewards[i], nexts[i], dones[i]
            #state_e = e[0]
            #action_e = e[1]
            #reward_e = e[2]
            #next_e = e[3]
            #done_e = e[4]
            #print("q target")
            
            q_target_next = self.critic(next_e, self.actor_target(next_e).detach().cpu().numpy()).detach().cpu().unsqueeze(1).numpy()
            #print("action_e")
            #print(action_e)
            q_target_next = reward_e.item() +(gamma * q_target_next[0].item())*(1-done_e.item())
            q_current = self.critic(state_e, action_e).detach().cpu().numpy()

            q_targets_next.append(q_target_next)
            q_currents.append(q_current[0].item())

            #calculate actor loss as average Q value from critic network (critic rates the actor)
            q_critic = self.critic(state_e, self.actor(state_e).detach().cpu().numpy()).detach().cpu().unsqueeze(1).numpy()
            q_critic = q_critic[0].item()
            q_critics.append(q_critic)
        critic_loss = F.mse_loss(torch.from_numpy(np.array(q_targets_next)),torch.from_numpy(np.array(q_currents))) 
        self.critic_optimizer.zero_grad()
        critic_loss.requires_grad = True
        critic_loss.backward()
        self.critic_optimizer.step() 
        
        actor_loss = -np.average(np.array(q_critic))
        actor_loss = torch.tensor(actor_loss)
        self.actor_optimizer.zero_grad()
        actor_loss.requires_grad = True
        actor_loss.backward()
        self.actor_optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(TAU)                     

    def soft_update(self,tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Modified to update both actor and critic
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
            target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

In [608]:

#model = CNN_Trackmania(image_height, image_width, num_images, keep_prob)
# Let us retrieve the TMRL Gymnasium environment.
# The environment you get from get_environment() depends on the content of config.json
env = get_environment()
agent = CNNAgent(state_size=6,action_size=3, seed=0) #State: speed, gear, rpm, x (4 images), act1, act2
                                                     #Action: speed, gear, rpm between -1 and 1
sleep(2)  # just so we have time to focus the TM20 window after starting the script


eps_start = 1
eps_end=0.01
eps_decay=0.995
scores = []
scores_window = deque(maxlen=100)  # last 100 scores
eps = eps_start
for i_episode in range(1, 2000):
    score = 0 #initialize episode score to 0
    obs, info = env.reset() #get state when lunar lander is restarted
    for _ in range(500):  # rtgym ensures this runs at 20Hz by default
        #act = model(torch.from_numpy(obs[3]))  # compute action
        #print(obs)
        #print(obs)
        #obs = np.asarray(obs).astype(np.float32)
        act = agent.act(obs, eps)
        #act = act.detach().numpy()
        
        act = np.array(act)
        action = (act + np.random.normal(0, 1, size=3)).clip(-1, 1)
        print(str(_) + ": " + str(act))
        next_obs, rew, terminated, truncated, info = env.step(act)  # step (rtgym ensures healthy time-steps)
        agent.step(obs, act, rew, next_obs, terminated) #step agent and learn reward from given action
        obs = next_obs
        score += rew
        if terminated or truncated:
            break
        
    scores_window.append(score)       # save most recent score
    scores.append(score)              # save most recent score
    eps = max(eps_end, eps_decay*eps) # decrease epsilon
    if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
    print("Episode " + str(i_episode) + ": " + str(score))
env.wait()  # rtgym-specific method to artificially 'pause' the environment when needed

0: [ 0.51590881 -0.15885684 -0.4821665 ]
1: [-0.19013173  0.56759718 -0.39337455]
2: [0.16676408 0.81622577 0.00937371]
3: [ 0.51160841  0.23673799 -0.49898732]
4: [0.96557095 0.62043447 0.8043319 ]
5: [0.4596635  0.79767658 0.36796786]
6: [-0.79859758 -0.13165633  0.22177395]
7: [ 0.93321274 -0.04598045  0.73061986]


c:\Users\Chris\AppData\Local\Programs\Python\Python37\lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.05256540000482346
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


8: [ 0.61005565  0.09739861 -0.9719166 ]
9: [-0.20235292  0.64968995  0.3363064 ]
10: [-0.01284427  0.73520555 -0.51217825]
11: [ 0.74094246 -0.61786582  0.13502148]
12: [ 0.9350805   0.60635894 -0.10406086]
13: [-0.35989079  0.01588129  0.86566765]
14: [0.10253449 0.41312282 0.09488182]
15: [0.08056721 0.92767709 0.20637126]
16: [-0.11002195  0.19257372 -0.23019771]
17: [-0.419341   -0.62121734 -0.62654094]
18: [ 0.31331878 -0.04693802 -0.82035128]
19: [0.75354074 0.84676203 0.68492045]
20: [ 0.84616488  0.08119985 -0.2174079 ]
21: [-0.44873176  0.62325742  0.69897193]
22: [0.17960237 0.89952975 0.15939002]
23: [0.32049076 0.99251568 0.83388244]
24: [-0.83525402  0.22556621 -0.0271116 ]
25: [ 0.69015515 -0.51392876  0.46297844]
26: [-0.55907893  0.58916594 -0.3349277 ]
27: [-0.79878496 -0.70728302  0.39534128]
28: [0.14773207 0.82003203 0.06839594]
29: [-0.94660641  0.26999982  0.21267684]
30: [-0.21758118 -0.25972012  0.9610333 ]
31: [-0.95672698  0.92206256 -0.63005612]
32: [-0.5788

c:\Users\Chris\AppData\Local\Programs\Python\Python37\lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.052225800031010294
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


50: [-0.48154615 -0.68462686  0.05514626]
51: [0.12280985 0.51096953 0.76775031]
52: [-0.37588351 -0.06621553  0.61809171]
53: [ 0.62482986 -0.62399741  0.99884072]
54: [-0.8330659   0.45110871  0.97364296]
55: [ 0.35703001 -0.36764573 -0.57295068]
56: [-0.99528487  0.64546282  0.05669195]
57: [-0.76219221  0.29853085  0.74730765]
58: [ 0.95703037 -0.79963862  0.70787622]
59: [-0.83730917 -0.45057231 -0.09404363]
60: [ 0.72271981 -0.73315889  0.04173106]
61: [-0.30589397  0.74372767 -0.44318037]
62: [-0.91867345  0.36199354  0.11671147]


...

83: [-0.53427379 -0.71622513 -0.09190386]


c:\Users\Chris\AppData\Local\Programs\Python\Python37\lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 0.06744710000202758
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


84: [-0.41337669 -0.05822492  0.90053666]
85: [-0.44605951  0.11636318  0.37640061]
86: [-0.10767123 -0.20244619  0.53528149]
87: [-0.50408466 -0.09310594  0.87420929]


c:\Users\Chris\AppData\Local\Programs\Python\Python37\lib\site-packages\rtgym\envs\real_time_env.py:379: UserWarning: Time-step timed out. Elapsed since last time-step: 19.194571299998643
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


88: [ 0.25427938  0.06275161 -0.178391  ]
Episode 1: 5.639999894425273
0: [-0.19317425  0.55710052  0.57635485]
1: [-0.25639135  0.25762181 -0.68586007]
2: [-0.23714449  0.18212495 -0.7209338 ]


In [ ]:
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space)
